In [1]:
import json
import scipy
import pandas as pd
import numpy as np
from collections import Counter
from openpyxl.utils import get_column_letter

In [2]:
# 自适应设置Excel列宽：https://laowangblog.com/pandas-openpyxl-excel-column-dimensions.html
def to_excel_auto_column_weight(df, writer, sheet_name):
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    column_widths = df.columns.to_series().apply(lambda x: len(x.encode('GB18030'))).values
    max_widths = df.astype(str).applymap(lambda x: len(x.encode('GB18030'))).agg(max).values
    widths = np.max([column_widths, max_widths], axis=0)
    worksheet = writer.sheets[sheet_name]
    for i, width in enumerate(widths, 1):
        worksheet.column_dimensions[get_column_letter(i)].width = width + 1


def to_excel(df, name, sheet_name):
    with pd.ExcelWriter(name) as writer:
        to_excel_auto_column_weight(df, writer, sheet_name=sheet_name)

In [3]:
mashup2api = {}
api2mashup = {}

with open("data/m-a_edges.csv", 'r', encoding='utf-8') as f:
    lines = f.readlines()
    header = 1
    for line in lines:
        if header:
            header = 0
            continue
        pair = line.split('\t')
        mashup = pair[0]
        api = pair[1][:-1]
        if mashup not in mashup2api.keys():
            mashup2api[mashup] = [api]
        else:
            mashup2api[mashup].append(api)
        if api not in api2mashup.keys():
            api2mashup[api] = [mashup]
        else:
            api2mashup[api].append(mashup)
mashup_stat = {"Mashup名称": [],
               "使用的API个数": [],
               "使用的API名称": []}
api_stat = {"API名称": [],
            "被Mashup使用的次数": [],
            "被使用的Mashup名称": []}
for mashup in mashup2api.keys():
    mashup_stat["Mashup名称"].append(mashup)
    mashup_stat["使用的API个数"].append(len(mashup2api[mashup]))
    mashup_stat["使用的API名称"].append(mashup2api[mashup])
mashup_df = pd.DataFrame(mashup_stat)
to_excel(mashup_df, "Mashup_to_API.xlsx", "Mashup中API的使用情况")
print("每个Mashup包含的Web API个数已统计完毕，结果已写入Mashup_to_API.xlsx")
for api in api2mashup.keys():
    api_stat["API名称"].append(api)
    api_stat["被Mashup使用的次数"].append(len(api2mashup[api]))
    api_stat["被使用的Mashup名称"].append(api2mashup[api])
api_df = pd.DataFrame(api_stat)
to_excel(api_df, "API_to_Mashup.xlsx", "API被Mashup使用的情况")
print("每个Web API被使用的次数已统计完毕，结果已写入API_to_Mashup.xlsx")

每个Mashup包含的Web API个数已统计完毕，结果已写入Mashup_to_API.xlsx
每个Web API被使用的次数已统计完毕，结果已写入API_to_Mashup.xlsx


In [4]:
print("Mashup中API的使用情况：")
mashup_df.describe()

Mashup中API的使用情况：


,使用的API个数
count,6071.000000
mean,2.077747
std,2.068645
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,37.000000


In [5]:
print("API被Mashup使用的情况：")
api_df.describe()

API被Mashup使用的情况：


,被Mashup使用的次数
count,1508.000000
mean,8.364721
std,60.375508
min,1.000000
25%,1.000000
50%,1.000000
75%,4.000000
max,1984.000000


In [6]:
print("使用API个数最多的前10个Mashup：")
mashup_df.sort_values(by=["使用的API个数"], ascending=False, ignore_index=True)[:10]

使用API个数最多的前10个Mashup：


,Mashup名称,使用的API个数,使用的API名称
0,Mashup: We-Wired Web,37,"[/api/flickr, /api/blogger, /api/delicious-0, ..."
1,Mashup: DoAt (do@),29,"[/api/flickr, /api/amazon-product-advertising,..."
2,Mashup: Pixelpipe,28,"[/api/flickr, /api/blogger, /api/buzznet, /api..."
3,Mashup: Sociotoco Search,24,"[/api/flickr, /api/feedburner, /api/blogger, /..."
4,Mashup: Gawkk.com,23,"[/api/bbc, /api/buzznet, /api/yahoo-video-sear..."
5,Mashup: vplan.com/search,22,"[/api/google-search, /api/google-maps, /api/ya..."
6,Mashup: coderbits,21,"[/api/twitter, /api/slideshare, /api/linkedin,..."
7,Mashup: ShipStation,21,"[/api/paypal, /api/fedex, /api/amazon-product-..."
8,Mashup: ConnectorLocal,21,"[/api/flickr, /api/upcomingorg, /api/eventful,..."
9,Mashup: What's Public,20,"[/api/flickr, /api/43things, /api/typepad, /ap..."


In [7]:
print("被Mashup使用最多的前10个API：")
api_df.sort_values(by=["被Mashup使用的次数"], ascending=False, ignore_index=True)[:10]

被Mashup使用最多的前10个API：


,API名称,被Mashup使用的次数,被使用的Mashup名称
0,/api/google-maps,1984,"[Mashup: BanksNearMe, Mashup: Poll Position, M..."
1,/api/twitter,671,"[Mashup: Hashtag Suggestions, Mashup: Online B..."
2,/api/youtube,562,"[Mashup: Mixblast, Mashup: Online Basketball C..."
3,/api/flickr,474,"[Mashup: DPhotoIndex, Mashup: Magic Mosaic, Ma..."
4,/api/facebook,381,"[Mashup: Online Basketball Camp, Mashup: Socce..."
5,/api/twilio,311,"[Mashup: The Gifanator, Mashup: UberSignal Cel..."
6,/api/amazon-product-advertising,304,"[Mashup: Penny Parrot, Mashup: Christmas List ..."
7,/api/lastfm,181,"[Mashup: Mixblast, Mashup: etrigg.com, Mashup:..."
8,/api/ebay,174,"[Mashup: ecomdash, Mashup: uShip on Cars.com, ..."
9,/api/twilio-sms,156,"[Mashup: Cloud Elements, Mashup: eSignature Ga..."


In [8]:
base_file_name = "data/raw/accessibility/api_accessibility/api_version_accessbiliby-"
api_endpoint = "API Endpoint"
homepage = "API Portal / Home Page"
sub_domain = ["com", "co", "ac", "org", "gov", "lk", "nc", "no"]

web2api = {}
web2api_mashup = {}
for i in range(1, 6):
    with open(base_file_name + str(i) + ".txt", 'r') as f:
        f = json.loads(f.read())
        for info in f:
            api = info.get("from_api").get("url")[:-1]
            provider = ""
            web_list = info.get("visit_status")
            if len(web_list) == 0:
                continue
            for web in web_list:
                if web.get("visit_label") == api_endpoint:
                    provider = web.get("visit_url")
            if len(provider) == 0:
                for web in web_list:
                    if web.get("visit_label") == homepage:
                        provider = web.get("visit_url")
            assert len(provider) != 0
            provider = provider.replace(' ', '')
            provider = provider.replace("https://", '')
            provider = provider.replace("http://", '')
            domains = provider.split('/')
            provider = '.'.join(domains[0].split('.')[-2:])
            if provider.split('.')[0] in sub_domain:
                provider = '.'.join(domains[0].split('.')[-3:])
            assert len(provider) != 0, print(i, api, domains)
            if provider not in web2api:
                web2api[provider] = [api]
            else:
                web2api[provider].append(api)
            if api in api2mashup.keys():
                if provider not in web2api_mashup:
                    web2api_mashup[provider] = [api]
                else:
                    web2api_mashup[provider].append(api)

web_stat = {"Web URL": [],
            "发布的API个数": [],
            "发布的API名称": []}
web_stat_mashup = {"Web URL": [],
            "发布的API个数": [],
            "发布的API名称": []}
for web in web2api.keys():
    web2api[web] = list(set(web2api[web]))
    web_stat["Web URL"].append(web)
    web_stat["发布的API个数"].append(len(web2api[web]))
    web_stat["发布的API名称"].append(web2api[web])
for web in web2api_mashup.keys():
    web2api_mashup[web] = list(set(web2api_mashup[web]))
    web_stat_mashup["Web URL"].append(web)
    web_stat_mashup["发布的API个数"].append(len(web2api_mashup[web]))
    web_stat_mashup["发布的API名称"].append(web2api_mashup[web])

web_df = pd.DataFrame(web_stat)
to_excel(web_df, "Web_Published_API.xlsx", sheet_name="Web发布的API情况")
print("每个Web API提供商发布的API个数已统计完毕，结果已写入Web_Published_API.xlsx")

web_mashup_df = pd.DataFrame(web_stat_mashup)
to_excel(web_mashup_df, "Web_Published_API_Mashup.xlsx", sheet_name="Web发布的有被Mashup使用的API情况")
print("每个Web API提供商发布的有被Mashup使用的API个数已统计完毕，结果已写入Web_Published_API_Mashup.xlsx")

每个Web API提供商发布的API个数已统计完毕，结果已写入Web_Published_API.xlsx
每个Web API提供商发布的有被Mashup使用的API个数已统计完毕，结果已写入Web_Published_API_Mashup.xlsx


In [9]:
print("Web API提供商发布的API情况：")
web_df.describe()

Web API提供商发布的API情况：


,发布的API个数
count,13757.000000
mean,1.554191
std,4.202691
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,195.000000


In [10]:
print("发布最多API的前10个提供商：")
web_df.sort_values(by=["发布的API个数"], ascending=False, ignore_index=True)[:10]

发布最多API的前10个提供商：


,Web URL,发布的API个数,发布的API名称
0,intrinio.com,195,"[/api/intrinio-nasdaq-tallinn-prices, /api/int..."
1,mashape.com,178,"[/api/poker-odds-calculator, /api/bnc-bitcoin-..."
2,googleapis.com,173,"[/api/google-cloud-dataproc, /api/google-civic..."
3,github.com,159,"[/api/github-repository-comments, /api/github-..."
4,google.com,122,"[/api/google-coordinate, /api/google-recaptcha..."
5,fusionfabric.cloud,102,"[/api/finastra-banks-information, /api/finastr..."
6,rapidapi.com,72,"[/api/unofficial-realtor, /api/unofficial-shaz..."
7,microsoft.com,67,[/api/microsoft-azure-cognitive-services-speec...
8,nasa.gov,65,"[/api/nasa-ssdcneos-sentry, /api/nasa-co2-virt..."
9,github.io,65,"[/api/apache-mesos-chronos, /api/viber-webhook..."


In [11]:
print("Web API提供商发布的有被Mashup使用的API情况：")
web_mashup_df.describe()

Web API提供商发布的有被Mashup使用的API情况：


,发布的API个数
count,1009.000000
mean,1.184341
std,1.613347
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,34.000000


In [12]:
print("发布最多有被Mashup使用的API的前10个提供商：")
web_mashup_df.sort_values(by=["发布的API个数"], ascending=False, ignore_index=True)[:10]

发布最多有被Mashup使用的API的前10个提供商：


,Web URL,发布的API个数,发布的API名称
0,googleapis.com,34,"[/api/google-play-developer, /api/google-assis..."
1,google.com,33,"[/api/google-openid, /api/google-recaptcha, /a..."
2,yahooapis.com,12,"[/api/yahoo-query-language-graphql, /api/yahoo..."
3,amazon.com,9,"[/api/amazon-ses, /api/amazon-alexa-skill-voic..."
4,mashape.com,9,"[/api/parsebot, /api/rxnlp-cluster-sentences-a..."
5,amazonaws.com,9,"[/api/amazon-fulfillment-web-service, /api/ama..."
6,microsoft.com,8,"[/api/microsoft-bing, /api/bing-traffic, /api/..."
7,yahoo.com,7,"[/api/yahoo-bbauth, /api/yahoo-internet-locati..."
8,ckan.net,5,"[/api/ckan-czech-republic, /api/ckan-norway, /..."
9,nytimes.com,5,"[/api/new-york-times-newswire, /api/new-york-t..."


In [13]:
cat2tag = {}
api2tag = {}
api2type = {}
cats = []
tags = []

mashup_api_cat_tag_stat = {"Mashup名称": [],
                           "对应Category": [],
                           "使用API对应的Tag统计": [],
                           "使用的API": []}
with open("data/raw/api_mashup/active_mashups_data.txt", 'r') as f:
    f = json.loads(f.read())
    for info in f:
        categories = list(set(info.get("categories")))
        mashup_api_cat_tag_stat["Mashup名称"].append(info.get("title"))
        mashup_api_cat_tag_stat["对应Category"].append(categories)
        apis = info.get("related_apis")
        api_list = []
        tag_list = []
        for api in apis:
            if not api:
                continue
            api2type[api.get("url")] = []
            api_list.append(api.get("url"))
            if len(api.get("tags")) == 0:
                api2tag[api.get("url")] = []
            for tag in set(api.get("tags")):
                tag_list.append(tag)
                tags.append(tag)
                if api.get("url") not in api2tag.keys():
                    api2tag[api.get("url")] = [tag]
                else:
                    api2tag[api.get("url")].append(tag)
            for ver in api.get("versions"):
                api2type[api.get("url")].append(ver.get("style"))
        for cat in categories:
            if cat not in cat2tag:
                cat2tag[cat] = [*tag_list]
            else:
                cat2tag[cat].extend(tag_list)
            cats.append(cat)
        tag_list = sorted(dict(Counter(tag_list)).items(), key=lambda x:x[1], reverse=True)
        mashup_api_cat_tag_stat["使用API对应的Tag统计"].append(tag_list)
        mashup_api_cat_tag_stat["使用的API"].append(api_list)

for api in api2type.keys():
    api2type[api] = set(api2type[api])
cats = sorted(dict(Counter(cats)).items(), key=lambda x:x[1], reverse=True)
tags = sorted(dict(Counter(tags)).items(), key=lambda x:x[1], reverse=True)

cat2tag_stat = {"Mashup Category": [],
                "关联API Tag": []}
for cat in cat2tag.keys():
    cat2tag[cat] = sorted(dict(Counter(cat2tag[cat])).items(), key=lambda x:x[1], reverse=True)
    cat2tag_stat["Mashup Category"].append(cat)
    cat2tag_stat["关联API Tag"].append(cat2tag[cat])

mashup_cat_tag_df = pd.DataFrame(mashup_api_cat_tag_stat)
to_excel(mashup_cat_tag_df, "Mashup_API_Category_Tag.xlsx", sheet_name="Mashup的Category与使用API的Tag对应情况")
print("每个Mashup的Category与其使用的API的Tag对应情况已统计完毕，结果已写入Mashup_API_Category_Tag.xlsx")

cat_tag_df = pd.DataFrame(cat2tag_stat)
to_excel(cat_tag_df, "Category_Tag.xlsx", sheet_name="Category与Tag对应情况")
print("每个Category与Tag对应情况已统计完毕，结果已写入Category_Tag.xlsx")

每个Mashup的Category与其使用的API的Tag对应情况已统计完毕，结果已写入Mashup_API_Category_Tag.xlsx
每个Category与Tag对应情况已统计完毕，结果已写入Category_Tag.xlsx


In [14]:
print("Mashup对应最多的五个类别：")
print(cats[:5])

Mashup对应最多的五个类别：
[('Mapping', 2285), ('Search', 964), ('Social', 944), ('eCommerce', 658), ('Photos', 656)]


In [15]:
print("API对应最多的五个功能标签：")
print(tags[:5])

API对应最多的五个功能标签：
[('Mapping', 3334), ('Viewer', 2393), ('Social', 2236), ('Search', 1411), ('Video', 1297)]


In [16]:
print("数量前五的Mashup类别相关联的数量最多五个的API功能标签：")
for cat, _ in cats[:10]:
    print(cat, cat2tag[cat][:5])

数量前五的Mashup类别相关联的数量最多五个的API功能标签：
Mapping [('Mapping', 2823), ('Viewer', 2143), ('Social', 411), ('Search', 286), ('Photos', 275)]
Search [('Search', 572), ('Social', 472), ('Mapping', 377), ('eCommerce', 323), ('Video', 304)]
Social [('Social', 1077), ('Blogging', 421), ('Webhooks', 380), ('Mapping', 374), ('Photos', 249)]
eCommerce [('eCommerce', 756), ('Search', 301), ('Advertising', 264), ('Mapping', 181), ('Auctions', 148)]
Photos [('Photos', 605), ('Video', 519), ('Mapping', 340), ('Social', 293), ('Viewer', 228)]
Video [('Video', 625), ('Media', 438), ('Social', 218), ('Mapping', 217), ('Search', 214)]
Travel [('Mapping', 561), ('Viewer', 405), ('Travel', 103), ('Photos', 96), ('Video', 95)]
Music [('Music', 502), ('Video', 245), ('Social', 213), ('Media', 195), ('Search', 102)]
Mobile [('Telephony', 340), ('Messaging', 199), ('Webhooks', 174), ('Cloud', 160), ('Mobile', 141)]
Messaging [('Telephony', 382), ('Messaging', 268), ('Webhooks', 197), ('Cloud', 189), ('Social', 177)]


In [17]:
api_pair = {}
for mashup in mashup2api.keys():
    if len(mashup2api[mashup]) <= 1:
        continue
    apis = mashup2api[mashup]
    for i in range(len(apis)):
        for j in range(i + 1, len(apis)):
            pair = [apis[i], apis[j]]
            pair = tuple(sorted(pair))
            if pair not in api_pair.keys():
                api_pair[pair] = 1
            else:
                api_pair[pair] += 1

api_pair = sorted(api_pair.items(), key=lambda x:x[1], reverse=True)
api_pair_stat = {"API-1": [],
                 "API-2": [],
                 "共同调用次数": [],
                 "是否存在相同Tag": [],
                 "API类型是否相同": []}
for pair, cnt in api_pair:
    api_pair_stat["API-1"].append(pair[0])
    api_pair_stat["API-2"].append(pair[1])
    api_pair_stat["共同调用次数"].append(cnt)
    if pair[0] not in api2tag.keys() or pair[1] not in api2tag.keys():
        api_pair_stat["是否存在相同Tag"].append(False)
    else:
        for tag in api2tag[pair[0]]:
            if tag in api2tag[pair[1]]:
                api_pair_stat["是否存在相同Tag"].append(True)
                break
        else:
            api_pair_stat["是否存在相同Tag"].append(False)
    if pair[0] not in api2type.keys() or pair[1] not in api2type.keys():
        api_pair_stat["API类型是否相同"].append(False)
        continue
    for tag in api2type[pair[0]]:
        if tag in api2type[pair[1]]:
            api_pair_stat["API类型是否相同"].append(True)
            break
    else:
        api_pair_stat["API类型是否相同"].append(False)


api_pair_df = pd.DataFrame(api_pair_stat)
to_excel(api_pair_df, "API_Pair.xlsx", sheet_name="API共同调用记录")
print("API间共同调用次数已统计完毕，结果已写入API_Pair.xlsx")

API间共同调用次数已统计完毕，结果已写入API_Pair.xlsx


In [18]:
print("共同调用次数前十的API Pair：")
api_pair_df.sort_values(["共同调用次数"], ascending=False, ignore_index=True)[:10]

共同调用次数前十的API Pair：


,API-1,API-2,共同调用次数,是否存在相同Tag,API类型是否相同
0,/api/facebook,/api/twitter,159,True,True
1,/api/google-maps,/api/twitter,154,False,True
2,/api/flickr,/api/youtube,140,True,True
3,/api/twilio,/api/twilio-sms,140,True,True
4,/api/google-maps,/api/youtube,132,False,True
5,/api/flickr,/api/google-maps,132,False,True
6,/api/facebook,/api/google-maps,97,False,True
7,/api/twitter,/api/youtube,96,False,True
8,/api/flickr,/api/twitter,96,False,True
9,/api/lastfm,/api/youtube,92,False,True


In [19]:
print("共同调用次数统计结果：")
api_pair_df.describe()

共同调用次数统计结果：


,共同调用次数
count,10557.000000
mean,1.874112
std,4.895649
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,159.000000


In [20]:
print("共同调用次数与API对是否存在相同Tag的关系：")
api_pair_df.groupby("是否存在相同Tag").mean()

共同调用次数与API对是否存在相同Tag的关系：


,共同调用次数,API类型是否相同
是否存在相同Tag,,
False,1.792291,0.723645
True,2.060964,0.790669


In [21]:
print("共同调用次数与API对类型是否相同的关系：")
api_pair_df.groupby("API类型是否相同").mean()

共同调用次数与API对类型是否相同的关系：


,共同调用次数,是否存在相同Tag
API类型是否相同,,
False,1.622872,0.249075
True,1.960535,0.323616


In [22]:
print("共同调用次数与API对是否存在相同Tag的相关系数：")
api_pair_df["共同调用次数"].corr(api_pair_df["是否存在相同Tag"], method="kendall")

共同调用次数与API对是否存在相同Tag的相关系数：


0.058659641774388094

In [23]:
print("共同调用次数与API对类型是否相同的相关系数：")
api_pair_df["共同调用次数"].corr(api_pair_df["API类型是否相同"], method="kendall")

共同调用次数与API对类型是否相同的相关系数：


0.0013151855658777285